# 필요 모듈 import

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# train-test set 만들기

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/foodrec/category_data_test.csv')
data

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,category
0,1,49302,1,1,Bulgarian Yogurt,120,16,112108,train,4,4,10,9,yogurt,dairy eggs,yogurt
1,816049,49302,7,1,Bulgarian Yogurt,120,16,47901,train,14,4,6,16,yogurt,dairy eggs,yogurt
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,2993,train,15,0,7,7,yogurt,dairy eggs,yogurt
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,41425,train,4,3,8,14,yogurt,dairy eggs,yogurt
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,187205,train,5,4,14,30,yogurt,dairy eggs,yogurt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447068,1713171,36616,1,1,Gin,124,5,6371,train,6,3,16,1,spirits,alcohol,alcohol
447069,1552485,47031,2,1,Kentucky Straight Bourbon Whiskey,124,5,205384,train,10,0,10,15,spirits,alcohol,alcohol
447070,1888595,47031,1,1,Kentucky Straight Bourbon Whiskey,124,5,86949,train,5,6,18,6,spirits,alcohol,alcohol
447071,2773302,234,1,0,Tennessee Whiskey,124,5,177868,train,4,6,14,30,spirits,alcohol,alcohol


In [ ]:
# 유저별 상품 구매 목록 : train/test set 분리를 위해 사용
product_list = data.groupby(['user_id'])['product_name'].agg({'unique'}).reset_index()
product_list.head()

,user_id,unique
0,1,"[Pistachios, Aged White Cheddar Popcorn, Cinna..."
1,2,"[Asian Pears, Organic Cashew Carrot Ginger Sou..."
2,5,"[Organic Raw Agave Nectar, Tamari Gluten Free ..."
3,7,"[Panama Peach Antioxidant Infusion, Antioxidan..."
4,8,"[Oyster Mushrooms, Vidalia Onions, Solid White..."


In [ ]:
# 상품명 중에 train set 구성
product_list['train'] = product_list['unique'].apply(lambda x: np.random.choice(x, len(x)//2))
product_list.head()

,user_id,unique,train
0,1,"[Pistachios, Aged White Cheddar Popcorn, Cinna...",[Aged White Cheddar Popcorn]
1,2,"[Asian Pears, Organic Cashew Carrot Ginger Sou...","[Chipotle Beef & Pork Realstick, Chicken Caesa..."
2,5,"[Organic Raw Agave Nectar, Tamari Gluten Free ...",[Organic Soba]
3,7,"[Panama Peach Antioxidant Infusion, Antioxidan...","[Vanilla Coffee Concentrate, Antioxidant Infus..."
4,8,"[Oyster Mushrooms, Vidalia Onions, Solid White...","[Vidalia Onions, Oyster Mushrooms, Vidalia Oni..."


In [ ]:
# test set 구성
test = []
for i in tqdm(range(0, product_list.shape[0])):
    test_rec = list(set(product_list.loc[i, 'unique']).difference(set(product_list.loc[i, 'train'])))
    test.append(test_rec) # uinque 상품 id - train 상품 id
product_list['test'] = test

  0%|          | 0/110227 [00:00<?, ?it/s]

In [ ]:
# train dataset 저장
train_map = product_list[['user_id', 'train']]

train_map_unnest = np.dstack((np.repeat(train_map.user_id.values, 
                            list(map(len, train_map.train))),np.concatenate(train_map.train.values)))

train_map_unnest = pd.DataFrame(data = train_map_unnest[0], columns = train_map.columns)
train_map_unnest.head()

,user_id,train
0,1,Aged White Cheddar Popcorn
1,2,Chipotle Beef & Pork Realstick
2,2,Chicken Caesar Salad
3,2,Chipotle Beef & Pork Realstick
4,2,Indian Paneer Tikka


In [ ]:
# test dataset 저장
test_map = product_list[['user_id', 'test']]

test_map_unnest = np.dstack((np.repeat(test_map.user_id.values,
                            list(map(len, test_map.test))),np.concatenate(test_map.test.values)))

test_map_unnest = pd.DataFrame(data = test_map_unnest[0], columns = test_map.columns)
test_map_unnest.head()

,user_id,test
0,1,Pistachios
1,1,Cinnamon Toast Crunch
2,2,Gluten Free Mushroom Risotto Bowl
3,2,Dark Chocolate Nuts & Sea Salt Bars
4,2,Shelled Pistachios


In [ ]:
train_map_unnest.columns = ['user_id', 'product_name']
test_map_unnest.columns = ['user_id', 'product_name']
train_map_unnest.to_csv('/content/drive/MyDrive/foodrec/train.csv', index=False)
test_map_unnest.to_csv('/content/drive/MyDrive/foodrec/test.csv', index=False)

# 추천시스템(Word2vec)

In [ ]:
# 저장한 데이터 및 train/test set 다시 불러오기
data = pd.read_csv('/content/drive/MyDrive/foodrec/category_data_test.csv')
train = pd.read_csv('/content/drive/MyDrive/foodrec/train.csv')
test = pd.read_csv('/content/drive/MyDrive/foodrec/test.csv')

In [ ]:
train['product_name'] = train['product_name'].astype(str)
test['product_name'] = test['product_name'].astype(str)
data['product_name'] = data['product_name'].astype(str)

In [ ]:
sol = test.groupby(['user_id'])['product_name'].agg({'unique'}).reset_index()
gt = {}
for user in tqdm(sol['user_id'].unique()): 
    gt[user] = list(sol[sol['user_id'] == user]['unique'].values[0])

  0%|          | 0/110227 [00:00<?, ?it/s]

In [ ]:
# product_name 별로 얼마나 구매했는지 보기 위해 df 생성
product_count = data['product_name'].value_counts().to_frame()
product_count = product_count.reset_index().rename(columns = {"index":"product_name","product_name":"count","count":"category"})
product_count

,product_name,count
0,Annie Peely Fruit 4.5 Z,155
1,Organic Brown Rice Cake Salt-Free,155
2,Hampshire 100% Natural Sour Cream,155
3,Gluten Free Sliced Mountain White Bread,155
4,Grassmilk 2% Reduced Fat Milk,155
...,...,...
13612,Organic Red Wine Vinegar,7
13613,Peanut Butter Chocolate Candy,7
13614,Corn Oil,7
13615,Imitation Crab Flakes,7


In [ ]:
# 각 상품의 id, 상품명, 카테고리, 구매횟수만 나타내게끔 df 생성
change_data = pd.merge(data, product_count, on = "product_name")
df = change_data.drop(columns=['add_to_cart_order', 'reordered','aisle_id','department_id','order_id','user_id','eval_set',
                               'order_number','order_dow','order_hour_of_day','days_since_prior_order','aisle','department'], axis=1)
df = df.drop_duplicates()
final_df = df.sort_values('count', ascending = False)
final_df

,product_id,product_name,category,count
175621,20169,Annie Peely Fruit 4.5 Z,snacks,155
159022,32052,Organic Brown Rice Cake Salt-Free,snacks,155
18792,3800,Hampshire 100% Natural Sour Cream,cheese,155
374929,28058,Gluten Free Sliced Mountain White Bread,bakery,155
37232,19820,Grassmilk 2% Reduced Fat Milk,milk,155
...,...,...,...,...
327475,31267,Organic Red Wine Vinegar,pantry,7
199308,21815,Peanut Butter Chocolate Candy,snacks,7
327539,6138,Corn Oil,pantry,7
403980,49436,Imitation Crab Flakes,packaged seafood,7


In [ ]:
# df 만든 이유: 유사성을 보이는 항목이 없거나 추천 개수에 모자를 때 추가해주기 위해서다른 사람들이 좋아한 상품들 df로 변환
popular_rec_model = final_df[0:500]
popular_rec_model

,product_id,product_name,category,count
175621,20169,Annie Peely Fruit 4.5 Z,snacks,155
159022,32052,Organic Brown Rice Cake Salt-Free,snacks,155
18792,3800,Hampshire 100% Natural Sour Cream,cheese,155
374929,28058,Gluten Free Sliced Mountain White Bread,bakery,155
37232,19820,Grassmilk 2% Reduced Fat Milk,milk,155
...,...,...,...,...
231923,38003,Sicilian Pistachio,ice cream ice,117
51679,37654,Butter with Canola Oil Spread,butter,117
43256,29118,French Vanilla Soy Creamer,soy lactosefree,117
373939,19261,Rocky Mountain Sourdough Bread,bakery,117


* 카테고리간의 유사도
* 상품명간의 유사도
* 상품 담은 순서를 통한 유사도

In [ ]:
train_user_product = train.groupby(['user_id'])['product_name'].agg({'unique'})
train_user_product.head()

,unique
user_id,
1,[Aged White Cheddar Popcorn]
2,"[Chipotle Beef & Pork Realstick, Chicken Caesa..."
5,[Organic Soba]
7,"[Vanilla Coffee Concentrate, Antioxidant Infus..."
8,"[Vidalia Onions, Oyster Mushrooms]"


In [ ]:
# string 형식으로 변환(int형식으로는 word2vec 학습 불가)
sentence = []
for user_sentence in train_user_product['unique'].values:
    sentence.append(list(map(str, user_sentence)))

In [ ]:
# Word2vec 학습 
from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence, size=100, window = 20, 
                           min_count=1, workers=4, iter=1000, sg=0)
'''
< parameter 설명 >
* size: vector 차원
* window: 타겟값을 중심으로 앞뒤 몇개의 단어를 볼 지
* min_count: 1번 등장한 단어는 무시
* worker: multiprocessing.cpu_count()
* iter:반복횟수(딥러닝에서의 epoch)
* sg: 1: skip gram 방식/ 0: CBOW 방식
'''

In [ ]:
# 값을 넣었을때 유사도 보기
embedding_model.wv.most_similar(positive=['Beef & Cheese Burritos'], topn=10)

[("Brown 'N Serve Original Fully Cooked Sausage Patties", 0.6395471692085266),
 ('Chimichanga, Steak & Cheese', 0.5572351217269897),
 ('Buffalo Style Chicken Wings', 0.5494890809059143),
 ('Ready to Eat Tuna Salad Original Deli Style', 0.5413788557052612),
 ('Sausage Pizza Rolls', 0.5225275754928589),
 ("Brown 'N Serve Turkey Sausage Links", 0.521355152130127),
 ('Meat Lovers Lasagna', 0.4727754592895508),
 ('Peas & Carrots', 0.4665820896625519),
 ('Spicy Mango With Jalapeño Smoked Chicken Sausage', 0.4442216753959656),
 ('Turkey Pot Pie', 0.42739957571029663)]

In [ ]:
#전체 상품에 대해 진행
total_rec_list = {}

bought_list = train.groupby(['user_id'])['product_name'].agg({'unique'}).reset_index()
for user in tqdm(train['user_id'].unique()):
    rec_list = []     
    bought = bought_list[bought_list['user_id'] == user]['unique'].values[0]
    word2vec_dict = {}
    for product in bought.tolist(): 
        for i in embedding_model.wv.most_similar(positive=[product], topn=20):
            if i[0] not in bought.tolist(): 
                if i[0] not in word2vec_dict.keys(): 
                    word2vec_dict[i[0]] = i[1]
                else:
                    word2vec_dict[i[0]] += i[1]
                
    rec_list = list(dict(sorted(word2vec_dict.items(), key = lambda x: x[1], reverse=True)).keys())
    total_rec_list[user] = rec_list[0:20]

  0%|          | 0/85705 [00:00<?, ?it/s]

In [ ]:
import six
import math

# https://github.com/kakao-arena/brunch-article-recommendation/blob/master/evaluate.py

class evaluate():
    def __init__(self, recs, gt, topn=100):
        self.recs = recs
        self.gt = gt 
        self.topn = topn 
        
    def _ndcg(self):
        Q, S = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            dcg = 0.0
            idcg = sum([1.0 / math.log(i + 2, 2) for i in range(min(len(seen), len(rec)))])
            for i, r in enumerate(rec):
                if r not in seen:
                    continue
                rank = i + 1
                dcg += 1.0 / math.log(rank + 1, 2)
            ndcg = dcg / idcg
            S += ndcg
            Q += 1
        return S / Q


    def _map(self):
        n, ap = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            _ap, correct = 0.0, 0.0
            for i, r in enumerate(rec):
                if r in seen:
                    correct += 1
                    _ap += (correct / (i + 1.0))
            _ap /= min(len(seen), len(rec))
            ap += _ap
            n += 1.0
        return ap / n


    def _entropy_diversity(self):
        sz = float(len(self.recs)) * self.topn
        freq = {}
        for u, rec in six.iteritems(self.recs):
            for r in rec:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent
    
    def _evaluate(self):
        print('MAP@%s: %s' % (self.topn, self._map()))
        print('NDCG@%s: %s' % (self.topn, self._ndcg()))
        print('EntDiv@%s: %s' % (self.topn, self._entropy_diversity()))

In [ ]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=9)
evaluate_func._evaluate()

MAP@9: 0.004033431107170592
NDCG@9: 0.008943833167913503
EntDiv@9: 19.006780762453296


In [ ]:
total_rec_list

{1: ['Pop-Tarts Frosted Strawberry/Frosted Brown Sugar Cinnamon Variety Pack Toaster Pastries',
  'Oatnut Bread',
  'Chunky Guacamole',
  'Dried Mangos',
  'Sea Salt Brown Rice Crackers',
  'Organic Creamy Almond Butter',
  'Dry Roasted Almonds',
  'Lindor Truffles',
  'Peppermint Patties',
  'Dry Roasted Macadamias with Sea Salt',
  'Shredded Beef Taquitos',
  'Full-Size Chocolate Bars',
  'Nutri Grain Bars Multi Pack',
  'Pane Turano Italian Bread',
  'Sweet Potato Chips',
  'Sharp Original Semisoft Cheese',
  'Organic Blueberries Package',
  'Raisin Bran',
  'Fresh Goat Cheese',
  'Turbinado Cane Sugar'],
 2: ['Cherry Tomato Medley',
  'G Series Perform Fierce Green Apple Sports Drink',
  'Vanilla Almond Crisp Protein Bar',
  'Organic Black Chia Seed',
  'Roast Turkey Breast & Stuffing Dinner',
  'Organic Grade B Maple Syrup',
  'Organic Cottage Cheese Blueberry Acai Chia',
  'Raspberry Zinger Caffeine Free Herbal Tea',
  'Organic Cottage Cheese Classic',
  'Organic Strawberry Chia 

In [ ]:
import joblib
# model save
joblib.dump(embedding_model,'/content/drive/MyDrive/foodrec/word2vec_pickle.pkl')

['/content/drive/MyDrive/foodrec/word2vec_pickle.pkl']

# flask 연결 test

In [ ]:
embedding_model.most_similar(positive=['Beef & Cheese Burritos'], topn=9)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[("Brown 'N Serve Original Fully Cooked Sausage Patties", 0.6395471692085266),
 ('Chimichanga, Steak & Cheese', 0.5572351217269897),
 ('Buffalo Style Chicken Wings', 0.5494890809059143),
 ('Ready to Eat Tuna Salad Original Deli Style', 0.5413788557052612),
 ('Sausage Pizza Rolls', 0.5225275754928589),
 ("Brown 'N Serve Turkey Sausage Links", 0.521355152130127),
 ('Meat Lovers Lasagna', 0.4727754592895508),
 ('Peas & Carrots', 0.4665820896625519),
 ('Spicy Mango With Jalapeño Smoked Chicken Sausage', 0.4442216753959656)]

In [ ]:
embedding_model.wv.similar_by_vector('Beef & Cheese Burritos', topn=10, restrict_vocab=None) 

[("Brown 'N Serve Original Fully Cooked Sausage Patties", 0.6395471692085266),
 ('Chimichanga, Steak & Cheese', 0.5572351217269897),
 ('Buffalo Style Chicken Wings', 0.5494890809059143),
 ('Ready to Eat Tuna Salad Original Deli Style', 0.5413788557052612),
 ('Sausage Pizza Rolls', 0.5225275754928589),
 ("Brown 'N Serve Turkey Sausage Links", 0.521355152130127),
 ('Meat Lovers Lasagna', 0.4727754592895508),
 ('Peas & Carrots', 0.4665820896625519),
 ('Spicy Mango With Jalapeño Smoked Chicken Sausage', 0.4442216753959656),
 ('Turkey Pot Pie', 0.42739957571029663)]

In [ ]:
embedding_model.wv.similar_by_vector('Beef & Cheese Burritos')

[("Brown 'N Serve Original Fully Cooked Sausage Patties", 0.6395471692085266),
 ('Chimichanga, Steak & Cheese', 0.557235062122345),
 ('Buffalo Style Chicken Wings', 0.5494891405105591),
 ('Ready to Eat Tuna Salad Original Deli Style', 0.5413788557052612),
 ('Sausage Pizza Rolls', 0.5225275754928589),
 ("Brown 'N Serve Turkey Sausage Links", 0.521355152130127),
 ('Meat Lovers Lasagna', 0.4727754592895508),
 ('Peas & Carrots', 0.4665820896625519),
 ('Spicy Mango With Jalapeño Smoked Chicken Sausage', 0.44422173500061035),
 ('Turkey Pot Pie', 0.42739951610565186)]

In [ ]:
import sys
print(sys.version)

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


모델을 pickle 파일로 저장해서 flask와 연동시키는데 error 발생함 -> 버전차이로 발생한 문제 -> 꼭 gensim, python version 확인해보기!